In [24]:
# ======================================
# 0. Core libs (DAY1~DAY9 공통)
# ======================================
import numpy as np
import pandas as pd

import shutil
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import matplotlib.font_manager as fm
from matplotlib import rcParams

# ======================================
# 1) Style FIRST (폰트보다 먼저)
# ======================================
plt.style.use("_mpl-gallery-nogrid")
sns.set_theme(style="white")
sns.set_style("ticks")
sns.set_context("notebook")

# ======================================
# 2) Font (NanumGothic) — style 적용 후 "고정"
# ======================================
FONT_DIR = Path(r"E:\Portfolio_ver1\seoul_pv_load_analysis\fonts")
FONT_PATH = FONT_DIR / "NanumGothic.ttf"

if not FONT_PATH.exists():
    raise FileNotFoundError(f"폰트 파일 없음: {FONT_PATH}")

fm.fontManager.addfont(str(FONT_PATH))
font_name = fm.FontProperties(fname=str(FONT_PATH)).get_name()

rcParams["font.family"] = font_name
rcParams["axes.unicode_minus"] = False

# (선택) 기본 글자 크기 통일
rcParams["font.size"] = 12
rcParams["axes.titlesize"] = 18
rcParams["axes.labelsize"] = 14
rcParams["xtick.labelsize"] = 12
rcParams["ytick.labelsize"] = 12

# 개별 객체용 (필요할 때만)
font_prop = fm.FontProperties(fname=str(FONT_PATH))

print("✅ Style+Font ready:", font_name)

✅ Style+Font ready: NanumGothic


In [5]:
'''
DAY 1 목표(최종):
    0) 원본 CSV를 먼저 백업한다 (증거 보존)
    1) 백업본(복제본)으로만 데이터를 로드한다
    2) USE_YM(YYYYMMDD) + USE_HM(HHMM) → datetime 생성
    3) hour / month 파생 변수 생성
    4) value(전력사용량) 숫자화
    5) 최소 분석 테이블 df_base 생성 + 성공 체크
'''

'\nDAY 1 목표(최종):\n    0) 원본 CSV를 먼저 백업한다 (증거 보존)\n    1) 백업본(복제본)으로만 데이터를 로드한다\n    2) USE_YM(YYYYMMDD) + USE_HM(HHMM) → datetime 생성\n    3) hour / month 파생 변수 생성\n    4) value(전력사용량) 숫자화\n    5) 최소 분석 테이블 df_base 생성 + 성공 체크\n'

### DAY 1: 데이터 구조 및 기초 확인

**목적**
- 서울시 전력 사용 데이터의 컬럼 구조 및 시간 정보 확인

**주요 작업**
- 시간 컬럼(USE_HM) 형식 확인
- 분석에 사용할 핵심 컬럼 정의

**산출물**
- 시간대(hour) 추출 로직 확정

In [25]:
# 1. 경로 세팅
# 1) 현재 노트북 실행 위치 기준으로 프로젝트 폴더를 찾는다.
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent # notebook의 부모가 프로젝트 루트라고 가정

# 2) data 폴더 경로
DATA_DIR = PROJECT_ROOT / "data"

# 3) data 폴더가 없으면 중단( 폴더 구조가 잘못된 것)
if not DATA_DIR.exists():
    raise FileNotFoundError(
        f"data 폴더를 찾지 못했음: {DATA_DIR}\n"
        "해결: 프로젝트 구조가 seoul_pv_load_analysis/data 형태 확인 요망" )

print("NOTEBOOK_DIR =", NOTEBOOK_DIR)
print("PROJECT_ROOT =", PROJECT_ROOT)
print("DATA_DIR =", DATA_DIR)

NOTEBOOK_DIR = e:\Portfolio_ver1\seoul_pv_load_analysis\notebooks
PROJECT_ROOT = e:\Portfolio_ver1\seoul_pv_load_analysis
DATA_DIR = e:\Portfolio_ver1\seoul_pv_load_analysis\data


In [26]:
# 2. 원본 CSV 찾기 (파일명 고정 버전)

# 1) 금일 사용할 원본 파일명
RAW_FILENAME = "법정동별시간별전력사용량.csv"
RAW_FILE = DATA_DIR / RAW_FILENAME

# 2) 원본 파일 존재 여부 확인
if not RAW_FILE.exists():
    # 파일이 없으면 data 폴더 내 파일 목록을 보여주고 중단한다.
    print("data 폴더 파일 목록:")
    for f in sorted(DATA_DIR.glob("*")):
        print("-", f.name)
    raise FileNotFoundError(
        f"원본 CSV를 찾지 못했음. {RAW_FILE}\n"
        "해결: data 폴더 아에 원본 파일이 있는지 확인 요망")

print("원본 CSV =", RAW_FILE)

원본 CSV = e:\Portfolio_ver1\seoul_pv_load_analysis\data\법정동별시간별전력사용량.csv


In [27]:
# 3. 원본 CSV 백업하기
# 핵심 원칙: 
# - 원본 CSV는 절대 수정 X
# - 분석 / 가공은 "복제본" 으로 진행

# 1) 백업 폴더 생성
BACKUP_DIR = DATA_DIR / "_backup"
BACKUP_DIR.mkdir(exist_ok = True)

# 2) 오늘 날짜 / 시간을 파일명에 찍어서 "스냅샷"으로 남긴다.
# - 같은 날 여러 번 백업해도 덮어쓰기 안 되게 하기 위함
ts = datetime.now().strftime("%Y - %m - %d_%H%M%S")
BACKUP_FILE = BACKUP_DIR / f"{RAW_FILE.stem} _ {ts}{RAW_FILE.suffix}"

# 3) 실제 파일 복사 수행
# - shutil.copy2는 메타데이터(수정시간 등)도 최대한 보존해준다.
shutil.copy2(RAW_FILE, BACKUP_FILE)

print("===백업 완료!===")
print(" -> 원본:", RAW_FILE.name)
print(" -> 백업:", BACKUP_FILE.name)

===백업 완료!===
 -> 원본: 법정동별시간별전력사용량.csv
 -> 백업: 법정동별시간별전력사용량 _ 2025 - 12 - 25_140138.csv


In [28]:
# 4. 백업본으로 로드 (이제부터 복제본만 사용)

# 1) 여러 인코딩을 시도해서 안정적으로 로드 한다.
encodings_to_try = ["utf-8-sig", "cp949", "euc-kr"]

df_raw = None
last_error = None

for enc in encodings_to_try:
    try:
        df_raw = pd.read_csv(BACKUP_FILE, encoding = enc)
        print(f"로드 성공 (encoding = {enc})")
        break
    except Exception as e:
        last_error = e

if df_raw is None:
    raise RuntimeError(
        "CSV 로드 실패. 인코딩 / 구분자 문제일 수 있음\n"
        f"마지막 에러: {last_error}")

# 2) 데이터 크기 / 샘플 확인
print("행 / 열 =", df_raw.shape)
display(df_raw.head(5))

로드 성공 (encoding = utf-8-sig)
행 / 열 = (9754804, 5)


,SIGUNGU_CD,BJDONG_CD,USE_YM,USE_HM,FDRCT_VLD_KWH
0,11650,10700,20220628,100,10782.0565
1,11650,10800,20220628,100,11394.8635
2,11650,10900,20220628,100,7273.9620
3,11740,10300,20220628,100,11008.8110
4,11710,11300,20220628,100,2905.1120


In [29]:
# 5. 컬럼 점검

# 1) 컬럼 목록 확인
print("===컬럼 목록===")
for c in df_raw.columns:
    print("-", c)

# 2) 결측치 비율(상위 15개) 확인
na_ratio = (df_raw.isna().mean() * 100).sort_values(ascending = False)
print("\n 결측치 비율 TOP 15 (%):")
display(na_ratio.head(15))

===컬럼 목록===
- SIGUNGU_CD
- BJDONG_CD
- USE_YM
- USE_HM
- FDRCT_VLD_KWH

 결측치 비율 TOP 15 (%):


SIGUNGU_CD       0.0
BJDONG_CD        0.0
USE_YM           0.0
USE_HM           0.0
FDRCT_VLD_KWH    0.0
dtype: float64

In [30]:
# 6. 날짜 / 시간 컬럼 처리

# 이 데이터의 시간 구조:
# - USE_YM: YYYYMMDD 형태 날짜(예: 20220628)
# - USE_HM: HHMM 형태 시간 (예: 100 -> 01:00, 2300 -> 23:00)
df = df_raw.copy()

# 1) USE_YM -> date(datetime)로 변환
df["date"] = pd.to_datetime(df["USE_YM"], format = "%Y%m%d", errors = "coerce")

# 2) USE_HM -> hour(0 ~ 23)로 변환
# - HHMM에서 앞의 HH만 쓰면 됨
df["hour"] = (df["USE_HM"] // 100 - 1).astype("Int64") # 결측 대응 Int64

# 3) date + hour -> datetime 생성
# - date가 NaT면 datetime도 NaT가 됨
df["datetime"] = df["date"] + pd.to_timedelta(df["hour"].fillna(0), unit = "h")

# 4) month 생성 
df["month"] = df["datetime"].dt.month

# 5) 생성 결과 샘플 확인
display(df[["USE_YM", "USE_HM", "date", "hour", "datetime", "month"]].head(10))

,USE_YM,USE_HM,date,hour,datetime,month
0,20220628,100,2022-06-28,0,2022-06-28,6
1,20220628,100,2022-06-28,0,2022-06-28,6
2,20220628,100,2022-06-28,0,2022-06-28,6
3,20220628,100,2022-06-28,0,2022-06-28,6
4,20220628,100,2022-06-28,0,2022-06-28,6
5,20220628,100,2022-06-28,0,2022-06-28,6
6,20220628,100,2022-06-28,0,2022-06-28,6
7,20220628,100,2022-06-28,0,2022-06-28,6
8,20220628,100,2022-06-28,0,2022-06-28,6
9,20220628,100,2022-06-28,0,2022-06-28,6


In [31]:
# 7. 시간축 품질 체크

# 1) date 변환 성공률
date_ok = df["date"].notna().mean() * 100
print(f"date 변환 성공률: {date_ok:.1f}%")

# 2) hour 범위 체크 (0 ~ 23)
hour_min = df["hour"].min()
hour_max = df["hour"].max()
print(f"hour 범위: {hour_min} ~ {hour_max}")

# 3) datetime 결측 확인
dt_ok = df["datetime"].notna().mean() * 100
print(f"datetime 생성 성공률: {dt_ok:.1f}%")

# 4) 혹시 0 ~ 23 밖이 나오면 데이터 / 파싱 문제 가능
if (hour_min is not pd.NA and hour_min < 0) or (hour_max is not pd.NA and hour_max > 23):
    print("hour가 0 ~ 23 범위를 벗어났으니 USE_HM 형식 다시 확인할 것")

date 변환 성공률: 100.0%
hour 범위: 0 ~ 23
datetime 생성 성공률: 100.0%


In [32]:
# 8. 값 칼럼 처리: FDRCT_VLD_KWH -> value

# 1) 전력 사용량 컬럼을 value로 통일하고 숫자로 변환
# - 콤마가 섞여 있을 수 있어 문자열 처리 후 to_numeric
df["value"] = pd.to_numeric(
    df["FDRCT_VLD_KWH"].astype(str).str.replace(",", ""), errors = "coerce")

# 2) value 품질 체크
value_ok = df["value"].notna().mean() * 100
print(f"value 숫자 변화 성공률: {value_ok:.1f}%")

display(df[["FDRCT_VLD_KWH", "value"]].head(10))

value 숫자 변화 성공률: 100.0%


,FDRCT_VLD_KWH,value
0,10782.0565,10782.0565
1,11394.8635,11394.8635
2,7273.9620,7273.9620
3,11008.8110,11008.8110
4,2905.1120,2905.1120
5,3416.9310,3416.9310
6,10473.6130,10473.6130
7,14196.5290,14196.5290
8,5894.3225,5894.3225
9,6393.6870,6393.6870


In [33]:
# 9. df_base 생성

# 1) DAY 1 최소 분석 테이블:
# - 이후 DAY 2 (시간대 구조 분석), DAY 3 (PV 패턴), DAY  4(겹침 분석)의 기반이 된다.
df_base = df[["datetime", "month", "hour", "value"]].copy()

print("df_base 크기 =", df_base.shape)
print("hour 범위  =", df_base["hour"].min(), "~", df_base["hour"].max())
print("value 결측률(%) =", df_base["value"].isna().mean() * 100)

display(df_base.head(10))

# DAY 1 종료
print("\n===DAY 1 Conclusion===")
print("- 원본 데이터는 백업 후 복제본으로 분석을 진행")
print("- USE_YM / USE_HM을 결합해 datetime을 생성했으며, hour / month 기반 분석 가능")
print("- 전력 사용량(value)을 수치형으로 확보하여 DAY 2 (시간대 부하 구조)로 진행할 수 있다.")

df_base 크기 = (9754804, 4)
hour 범위  = 0 ~ 23
value 결측률(%) = 0.0


,datetime,month,hour,value
0,2022-06-28,6,0,10782.0565
1,2022-06-28,6,0,11394.8635
2,2022-06-28,6,0,7273.9620
3,2022-06-28,6,0,11008.8110
4,2022-06-28,6,0,2905.1120
5,2022-06-28,6,0,3416.9310
6,2022-06-28,6,0,10473.6130
7,2022-06-28,6,0,14196.5290
8,2022-06-28,6,0,5894.3225
9,2022-06-28,6,0,6393.6870



===DAY 1 Conclusion===
- 원본 데이터는 백업 후 복제본으로 분석을 진행
- USE_YM / USE_HM을 결합해 datetime을 생성했으며, hour / month 기반 분석 가능
- 전력 사용량(value)을 수치형으로 확보하여 DAY 2 (시간대 부하 구조)로 진행할 수 있다.
